1) Connect workspace and create experiment

In [28]:
#creating the azure ml compute cluster
from azureml.core import Workspace, Experiment, Datastore, Dataset, ScriptRunConfig

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.run import Run

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform

from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

from azureml.widgets import RunDetails

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import argparse
import joblib
import numpy as np
import pandas as pd
import os

ws = Workspace.from_config() # this automatically looks for a directory .azureml

experiment = Experiment(workspace=ws, name="bankmkt-experiment")

# Choose a name for your CPU cluster
cpu_cluster_name = "computer-instance"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_v2',
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [11]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-133557
Azure region: southcentralus
Resource group: aml-quickstarts-133557


In [12]:
#checking the azure ml sdk version
from azureml.core import VERSION

print ('Version: ' + VERSION)

Version: 1.19.0


In [30]:
ps = RandomParameterSampling({'--C': uniform(0.1, 1),
                              '--max_iter': quniform(100, 1500, 100),})
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("./scripts",
              compute_target=cpu_cluster,
              entry_script="logit_train.py" )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
     hyperparameter_sampling=ps,
     policy=policy,
     primary_metric_name='Accuracy',
     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
     max_total_runs=25,
     max_concurrent_runs=4,)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [31]:
# data

datastore = ws.get_default_datastore()

factory = TabularDatasetFactory()

data_path_train = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_path_valid = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_validate.csv"
data_path_test = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv"

ds_train = factory.from_delimited_files(data_path_train)
ds_valid = factory.from_delimited_files(data_path_valid)
ds_test = factory.from_delimited_files(data_path_test)

In [25]:
x, y = clean_data(ds)

TypeError: 'NoneType' object is not iterable

In [26]:
run = Run.get_context()

# Evaluation of model perf on our holdout-set.

def clean_data(data):
    # Dict for cleaning data
    months = {"jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6, "jul": 7, "aug": 8, "sep": 9, "oct": 10,
              "nov": 11, "dec": 12}
    weekdays = {"mon": 1, "tue": 2, "wed": 3, "thu": 4, "fri": 5, "sat": 6, "sun": 7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)

    return x_df, y_df
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
    

def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

    args = parser.parse_args()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))

    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))

if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--C C] [--max_iter MAX_ITER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/azureuser/.local/share/jupyter/runtime/kernel-37e0a6a3-9b74-497b-9c61-b33d07093e97.json


SystemExit: 2